In [1]:
from gurobipy import *

# 創建模型
m = Model("Robot and Cargo Path Planning")

# 定義網格大小和時間上限
grid_size, T, num_robots, num_cargos = 3, 8, 7, 7

# 變量
x = m.addVars(num_robots, grid_size, grid_size, T+1, vtype=GRB.BINARY, name="robot")  # 機器人的位置變量
c = m.addVars(num_cargos, grid_size, grid_size, T+1, vtype=GRB.BINARY, name="cargo")  # 貨物的位置變量
z = m.addVars(num_robots, num_cargos, T+1, vtype=GRB.BINARY, name="carrying")         # 機器人搬運貨物的變量
y = m.addVars(num_cargos, T+1, vtype=GRB.BINARY, name="finish")                       # 貨物到達終點的變量
#機器人移動方向變量
move = m.addVars(num_robots, grid_size, grid_size, 4, T, vtype=GRB.BINARY, name="move")  # 0: 左, 1: 右, 2: 上, 3: 下
#最慢到達貨物的到達時間
max_arrival_time = m.addVar(vtype=GRB.INTEGER, name="max_arrival_time")


# 權重參數
alpha = 0.001  
# 設定初始位置和目標位置
initial_positions = [(0, 2, 2), (1, 0, 2),(2 ,1, 0), (3, 0, 0), (4, 0, 1), (5, 1, 1), (6, 2, 0)]  # 機器人的初始位置
cargo_init_pos = [(0, 2, 2), (1, 0, 2),(2 ,1, 0), (3, 0, 0), (4, 0, 1), (5, 1, 1), (6, 2, 0)]  # 貨物的初始位置
cargo_goal_pos = [(0, 0, 2), (1, 1, 0),(2 ,1, 2), (None, None, None), (None, None, None), (None, None, None), (None, None, None)]  # 貨物的目標位置，(None, None, None) 表示沒有固定終點

# 目標函數，最小化貨物的到達時間和所有機器人的移動總次數
m.setObjective(
    max_arrival_time + 
    alpha * quicksum(x[r, i, j, t] * (1 - x[r, i, j, t-1]) 
                     for r in range(num_robots) 
                     for i in range(grid_size) 
                     for j in range(grid_size) 
                     for t in range(1, T+1)),
    GRB.MINIMIZE
)

# 最慢到達貨物的到達時間
for c_idx in range(num_cargos):
    if cargo_goal_pos[c_idx][0] is not None:
        for t in range(1, T+1):
            m.addConstr(max_arrival_time >= t * (y[c_idx, t] - y[c_idx, t-1]))


for r, i, j in initial_positions:
    m.addConstr(x[r, i, j, 0] == 1)

# 貨物的初始位置
for c_idx, i, j in cargo_init_pos:
    m.addConstr(c[c_idx, i, j, 0] == 1)

# 貨物的目標位置（只對有固定終點的貨物設置約束）
for c_idx, goal in enumerate(cargo_goal_pos):
    if goal[0] is not None:
        _, i, j = goal
        m.addConstr(y[c_idx, T] == 1)  # 確保在最後時刻 T 時，有固定終點的貨物都到達終點
        for t in range(1, T+1):
            m.addConstr(y[c_idx, t] <= c[c_idx, i, j, t])
            m.addConstr(y[c_idx, t] >= y[c_idx, t-1])  # 一旦到達終點，狀態保持到達

# 每個格子在每個時刻只能有一個機器人或貨物
for t in range(T+1):
    for i in range(grid_size):
        for j in range(grid_size):
            m.addConstr(quicksum(x[r, i, j, t] for r in range(num_robots)) <= 1)  # 機器人
            m.addConstr(quicksum(c[cargo, i, j, t] for cargo in range(num_cargos)) <= 1)  # 貨物

# 機器人和貨物在每個時刻只會在一個格子
for t in range(T+1):
    for r in range(num_robots):
        m.addConstr(quicksum(x[r, i, j, t] for i in range(grid_size) for j in range(grid_size)) == 1)
    for cargo in range(num_cargos):
        m.addConstr(quicksum(c[cargo, i, j, t] for i in range(grid_size) for j in range(grid_size)) == 1)

# 機器人和貨物的移動
for t in range(T):
    for r in range(num_robots):  #機器人只能在相鄰位置之間移動
        for i in range(grid_size):
            for j in range(grid_size):
                m.addConstr(x[r, i, j, t+1] <= x[r, i, j, t] + 
                            quicksum(x[r, i1, j1, t] 
                                     for i1, j1 in [(i-1, j), (i+1, j), (i, j-1), (i, j+1)]
                                     if 0 <= i1 < grid_size and 0 <= j1 < grid_size))

    for cargo in range(num_cargos):   #貨物的移動需要由機器人來完成，並且只能在相鄰位置之間移動
        for i in range(grid_size):
            for j in range(grid_size):
                m.addConstr(c[cargo, i, j, t+1] <= c[cargo, i, j, t] + 
                            quicksum(z[r, cargo, t] * x[r, i, j, t+1] for r in range(num_robots)))

# 機器人搬運貨物的約束
for t in range(T):
    for r in range(num_robots):
        for cargo in range(num_cargos):
            m.addConstr(z[r, cargo, t] <= quicksum(x[r, i, j, t] * c[cargo, i, j, t] 
                                     for i in range(grid_size) for j in range(grid_size)))

# 機器人和貨物搬運數量的限制
for t in range(T+1):
    for r in range(num_robots):
        m.addConstr(quicksum(z[r, cargo, t] for cargo in range(num_cargos)) <= 1)  # 每個機器人同時只能搬運一個貨物
    for cargo in range(num_cargos):
        m.addConstr(quicksum(z[r, cargo, t] for r in range(num_robots)) <= 1)  # 每個貨物同時只能被一個機器人搬運

# 機器人的移動和位置變化
for t in range(T):
    for r in range(num_robots):
        for i in range(grid_size):
            for j in range(grid_size):
                # 左移
                if j > 0:
                    m.addConstr(move[r, i, j, 0, t] == x[r, i, j, t] * x[r, i, j-1, t+1])
                # 右移
                if j < grid_size - 1:
                    m.addConstr(move[r, i, j, 1, t] == x[r, i, j, t] * x[r, i, j+1, t+1])
                # 上移
                if i > 0:
                    m.addConstr(move[r, i, j, 2, t] == x[r, i, j, t] * x[r, i-1, j, t+1])
                # 下移
                if i < grid_size - 1:
                    m.addConstr(move[r, i, j, 3, t] == x[r, i, j, t] * x[r, i+1, j, t+1])
                
                
# 避免同一時間內格點的水平移入和垂直移出同時發生，或反之
for t in range(T):
    for i in range(grid_size):
        for j in range(grid_size):
            # 水平移入 (從左邊或右邊進入)
            horizontal_in = quicksum(move[r, i, j, 0, t] for r in range(num_robots)) + \
                            quicksum(move[r, i, j, 1, t] for r in range(num_robots))
            
            # 水平移出 (移到左邊或右邊)
            horizontal_out = quicksum(move[r, i, j-1, 1, t] for r in range(num_robots) if j > 0) + \
                             quicksum(move[r, i, j+1, 0, t] for r in range(num_robots) if j < grid_size - 1)

            # 垂直移入 (從上方或下方進入)
            vertical_in = quicksum(move[r, i, j, 2, t] for r in range(num_robots)) + \
                          quicksum(move[r, i, j, 3, t] for r in range(num_robots))

            # 垂直移出 (移到上方或下方)
            vertical_out = quicksum(move[r, i-1, j, 3, t] for r in range(num_robots) if i > 0) + \
                           quicksum(move[r, i+1, j, 2, t] for r in range(num_robots) if i < grid_size - 1)

            # 避免水平移入與垂直移出同時發生
            m.addConstr(horizontal_in + vertical_out <= 1)
            
            # 避免垂直移入與水平移出同時發生
            m.addConstr(vertical_in + horizontal_out <= 1)

# 避免機器人直接互换位置
for t in range(T):
    for i in range(grid_size):
        for j in range(grid_size):
            for di, dj in [(0, 1), (1, 0)]:  
                if i + di < grid_size and j + dj < grid_size:
                    m.addConstr(quicksum(x[r, i, j, t] * x[r, i+di, j+dj, t+1] for r in range(num_robots)) +
                                quicksum(x[r, i+di, j+dj, t] * x[r, i, j, t+1] for r in range(num_robots)) <= 1)

# 機器人轉向移動限制
for t in range(T-1):
    for r in range(num_robots):
        for i in range(grid_size):
            for j in range(grid_size):
                # 如果在t時刻垂直移動（上移或下移），則在t+1時刻不能水平移動
                vertical_move_t = move[r, i, j, 2, t] + move[r, i, j, 3, t]
                horizontal_move_t1 = quicksum(move[r, i1, j1, d, t+1] 
                                              for i1 in range(grid_size) 
                                              for j1 in range(grid_size) 
                                              for d in [0, 1])  # 0: 左, 1: 右
                m.addConstr(vertical_move_t + horizontal_move_t1 <= 1)

                # 如果在t時刻水平移動（左移或右移），則在t+1時刻不能垂直移動
                horizontal_move_t = move[r, i, j, 0, t] + move[r, i, j, 1, t]
                vertical_move_t1 = quicksum(move[r, i1, j1, d, t+1] 
                                            for i1 in range(grid_size) 
                                            for j1 in range(grid_size) 
                                            for d in [2, 3])  # 2: 上, 3: 下
                m.addConstr(horizontal_move_t + vertical_move_t1 <= 1)


m.setParam('MIPFocus', 2) 
m.optimize()


# 輸出結果
if m.status == GRB.OPTIMAL:
    for t in range(T+1):
        print(f"time period {t}")
        
        grid = [[0 for _ in range(grid_size)] for _ in range(grid_size)]
        
        for r in range(num_robots):
            for i in range(grid_size):
                for j in range(grid_size):
                    if x[r, i, j, t].x > 0.5:
                        grid[i][j] = r + 1  
        
        for row in grid:
            print("|", end=" ")
            for cell in row:
                print(f"{cell:2}", end=" ")
            print("|")
        print()  
        for r in range(num_robots):
            robot_pos = [(i+1, j+1) for i in range(grid_size) for j in range(grid_size) if x[r, i, j, t].x > 0.5]
            print(f"機器人{r+1}在 {robot_pos}", end="")
            
            carrying = [cargo+1 for cargo in range(num_cargos) if z[r, cargo, t].x > 0.5]
            if carrying:
                print(f", 正在搬運貨物{carrying[0]}", end="")
            print()
        # 貨物位置資訊
        for cargo in range(num_cargos):
            cargo_pos = [(i+1, j+1) for i in range(grid_size) for j in range(grid_size) if c[cargo, i, j, t].x > 0.5]
            if cargo_goal_pos[cargo][0] is not None:
                _, goal_i, goal_j = cargo_goal_pos[cargo]
                goal_i, goal_j = goal_i + 1, goal_j + 1
                if cargo_pos == [(goal_i, goal_j)]:
                    print(f"貨物{cargo+1}在 {cargo_pos}, 貨物已到達目標位置")
                else:
                    print(f"貨物{cargo+1}在 {cargo_pos}")
            else:
                print(f"貨物{cargo+1}在 {cargo_pos} (無固定目標位置)")
        print() 
else:
    print("無法找到解")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-10
Set parameter MIPFocus to value 2
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1185G7 @ 3.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 2033 rows, 3655 columns and 26687 nonzeros
Model fingerprint: 0x112c8b43
Model has 504 quadratic objective terms
Model has 2336 quadratic constraints
Variable types: 0 continuous, 3655 integer (3654 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e+00, 1e+00]
  Objective range  [1e-03, 1e+00]
  QObjective range [2e-03, 2e-03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
  QRHS range       [1e+00, 1e+00]
Presolve removed 1040 rows and 1524 columns
Presolve time: 0.08s
Presolved: 16180 rows, 8362